# modAL + pyhard- Comparando estratégias

- modAL

    - Amostra por incerteza
    - Amostragem aleatória
    - Consulta por comitê
    - Aprendizado passivo
    - Redução do erro esperado

- Pyhard
    - Harmfulness
    - Usefulness
    - H+U
    - Local Set Cardinality 
    - N2
    - F3

In [1]:
%run -i set_environment

## Bibliotecas

In [2]:
%run -i importing_libraries

## Classificadores

### Algoritmos

In [3]:
%run -i classifiers

### Conjunto de dados

In [4]:
%run -i importing_datasets

## Estatratégias

## modAL

In [5]:
def modal_framework(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy):

    sample_size = 0 #contador de amostras utilizadas pela estratégia
    accuracy_history = []
    f1_history = []
    start = timer()    
    
    print(strategy[1])
    if (strategy[1] == "query_by_committee"):
        learner_list = []
        for j in range(1, cost+1): # Loop para criação do comitê

            X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
            sample_size = sample_size + len(X_train)
            
            # initializing learner
            learner = ActiveLearner(
                estimator= which_classifier(classifier),
                X_training = X_train, y_training = y_train 
            )
            learner_list.append(learner)
            
        # assembling the committee
        committee = Committee(
            learner_list=learner_list,
            query_strategy=vote_entropy_sampling)
            
    else: 
        X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])

        sample_size = sample_size + len(X_train)
        committee = None

        if (strategy[1] != "random_sampling"):
            learner = ActiveLearner (
                estimator= which_classifier(classifier), #cls,
                query_strategy=uncertainty_sampling,
                X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
            )
        else:
            learner = None
     
    if (strategy[1] != "random_sampling"):
        accuracy_history.append(learner.score(X_pool, y_pool))
        f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
    
    for i in range(cost):
        sample_size, accuracy_history, f1_history = eval(strategy[1] + "(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_pool, y_train, y_pool, learner, committee)")
    
    end = timer()
    time_elapsed = end - start

    return {"accuracy_history": accuracy_history,
         "f1_history": f1_history,
         "package": "modAL",
         "time_elapsed": time_elapsed,
         "classifier": classifier,
         "sample_size": sample_size / len(X_raw),
         "strategy": strategy[0]}

### Amostra por incerteza

In [6]:
def uncertain_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_test, y_train, y_test, learner, committee = None):
      
    idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]

    if classifier_uncertainty(learner, X_train[0].reshape(1,-1)) > 0.2:
        #print("IF", learner.score(X_test, y_test))
        sample_size = sample_size + len(X_train)
        learner.teach(X_train, y_train)
    accuracy_history.append(learner.score(X_test, y_test))
    f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))
        
    return sample_size, accuracy_history, f1_history

### Amostragem aleatória

In [7]:
def random_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_test, y_train, y_test, learner = None, committee = None):

    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    sample_size = sample_size + len(X_train)

    cls = which_classifier(classifier)
    cls.fit(X_train, y_train)

    accuracy_history.append(cls.score(X_test,y_test))
    f1_history.append(compute_f1(cls, X_test, y_test, "weighted"))
        
    return sample_size, accuracy_history, f1_history

### Consulta por comitê

In [8]:
def query_by_committee(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_pool, y_train, y_pool, learner, committee):    

    # print("\t Size of X_pool:", len(X_pool))
    query_idx, query_instance = committee.query(X_pool, n_instances = init_size)
    sample_size = sample_size + len(query_idx)

    committee.teach(
        X = X_pool[query_idx],
        y = y_pool[query_idx]
    )

    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx)

    accuracy_history.append(committee.score(X_pool, y_pool))
    f1_history.append(compute_f1(committee, X_pool, y_pool, "weighted"))
    
    return sample_size, accuracy_history, f1_history

### Expected Error Reduction

In [9]:
def exp_error_reduction(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_pool, y_train, y_pool, learner, committee = None):

    # print("\t Size of X_pool:", len(X_pool))
    exp_error_idx = expected_error_reduction(learner, X_pool, 'binary', n_instances=init_size)

    learner.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
    sample_size = sample_size + init_size

    # X_pool = np.delete(X_pool, exp_error_idx, axis=0)
    # y_pool = np.delete(y_pool, exp_error_idx)

    accuracy_history.append(learner.score(X_pool, y_pool))
    f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
    
    return sample_size, accuracy_history, f1_history

### Expected Model Change

In [10]:
def exp_model_change(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, X_train, X_pool, y_train, y_pool, learner, committee = None):

    # print("\t Size of X_pool:", len(X_pool))
    exp_error_idx = np.random.choice(range(len(X_pool)), size=init_size, replace=False)
    aux = deepcopy(learner)

    aux.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
    score_aux = aux.score(X_pool, y_pool)
    score_learner = learner.score(X_pool, y_pool)

    if score_aux > score_learner:
        learner = deepcopy(aux)
        sample_size = sample_size + init_size

    X_pool = np.delete(X_pool, exp_error_idx, axis=0)
    y_pool = np.delete(y_pool, exp_error_idx, axis=0)

    accuracy_history.append(learner.score(X_pool, y_pool))
    f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))

    return sample_size, accuracy_history, f1_history

## Setup

In [11]:
%run -i setup_functions

## Pyhard Strategies

In [12]:
def pyhard_strategies(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy):
    
    from modAL.uncertainty import classifier_uncertainty
    
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    accuracy_history = []
    f1_history = []
    start = timer()
    
    strategy = config(strategy)
    
    # parte randomica inicial da estratégia
    
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    
    sample_size = sample_size + len(X_train)

    learner = ActiveLearner (
        estimator= which_classifier(classifier), #cls,
        query_strategy=uncertainty_sampling,
        X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
    )

    accuracy_history.append(learner.score(X_test, y_test))
    f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))

    total_of_samples = 1

    #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)

    idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]

    X_rawAndY_raw = np.column_stack([X_raw[idx_data[idx_bag][TRAIN]],y_raw[idx_data[idx_bag][TRAIN]]])
    np.savetxt("data.csv", X_rawAndY_raw, fmt='%i', delimiter=",")
    
    which_pyhard_measure(strategy['measure'])

    !pyhard --no-isa

    df = pd.read_csv('metadata.csv')

    idx = list(df.sort_values(by=strategy['sortby'], ascending=strategy['ascending'])['instances'][:cost])

    X_train = X_raw[idx_data[idx_bag][TRAIN][idx]]
    y_train = y_raw[idx_data[idx_bag][TRAIN][idx]]

    sample_size = cost
    learner.teach(X_train, y_train)
    
    accuracy_history.append(learner.score(X_test, y_test))
    f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))
    
    end = timer()
    time_elapsed = end - start

    return { "accuracy_history": accuracy_history,
         "f1_history": f1_history,
         "package": "Pyhard",
         "time_elapsed": time_elapsed,
         "classifier": classifier,
         "sample_size": sample_size / len(X_raw),
         "strategy": strategy['name']}

In [13]:
datasets = os.listdir('./datasets/luis')
classifiers = ['5NN', 'C4.5', 'NB','RF']

In [14]:
total_performance_history = []

In [15]:
datasets

['1043_ada_agnostic.arff',
 '1048_jEdit_4.2_4.3.arff',
 '1049_pc4.arff',
 '1050_pc3.arff',
 '1056_mc1.arff',
 '1065_kc3.arff',
 '1068_pc1.arff',
 '1069_pc2.arff',
 '1071_mw1.arff',
 '1100_PopularKids.arff',
 '1116_musk.arff',
 '1167_pc1_req.arff',
 '11_balance-scale.arff',
 '12_mfeat-factors.arff',
 '1460_banana.arff',
 '1462_banknote-authentication.arff',
 '1464_blood-transfusion-service-center.arff',
 '1466_cardiotocography.arff',
 '1467_climate-model-simulation-crashes.arff',
 '1475_first-order-theorem-proving.arff',
 '1479_hill-valley.arff',
 '1480_ilpd.arff',
 '1485_madelon.arff',
 '1487_ozone-level-8hr.arff',
 '1489_phoneme.arff',
 '1494_qsar-biodeg.arff',
 '1496_ringnorm.arff',
 '1497_wall-robot-navigation.arff',
 '1498_sa-heart.arff',
 '14_mfeat-fourier.arff',
 '1501_semeion.arff',
 '1504_steel-plates-fault.arff',
 '1506_thoracic-surgery.arff',
 '1507_twonorm.arff',
 '1508_user-knowledge.arff',
 '1510_wdbc.arff',
 '1511_wholesale-customers.arff',
 '1523_vertebra-column.arff',
 

In [16]:
metadata = []

for ds in datasets:
    metadata.append(fetch_datasets(ds))

metadata = pd.DataFrame.from_dict(metadata)
metadata

,name,instances,classes,attributes,nominal attributes,majority,minority
0,1043_ada_agnostic,4562,2,47,1,75.19,24.81
1,1048_jEdit_4.2_4.3,369,2,8,1,55.28,44.72
2,1049_pc4,1458,2,37,1,87.79,12.21
3,1050_pc3,1563,2,37,1,89.76,10.24
4,1056_mc1,9466,2,38,1,99.28,0.72
...,...,...,...,...,...,...,...
95,54_vehicle,846,4,18,1,25.77,23.52
96,59_ionosphere,351,2,33,1,64.10,35.90
97,60_waveform-5000,5000,3,40,1,33.84,33.06
98,679_rmftsa_sleepdata,1024,4,2,1,39.45,9.18


In [17]:
pyhard_strategies_names = ['H','U','H+U','LSC','N2','F3']

for ds in datasets:
    for classifier in classifiers:
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
        #para cada i em idx_bag ("n_splits") (1 a 5)
        for idx_bag in range(n_splits):
            for ph_strategy in pyhard_strategies_names:
                tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " " + ph_strategy)
                result = pyhard_strategies(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, init_size, cost, ph_strategy)
                result['dataset'] = ds[:-5]
                total_performance_history.append(result)
                tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag) + "/" + str(n_splits) + " " + ph_strategy)        

Testando: 1043_ada_agnostic 5NN 0/5 H
{'name': 'Lowest Harmfulness Sampling', 'measure': 'Harmfulness', 'sortby': ['feature_Harmfulness'], 'ascending': [True]}
^C


KeyError: 'feature_Harmfulness'

In [18]:
modal_strategies = {"Uncertain Sampling":"uncertain_sampling", "Random Sampling":"random_sampling", "Query by Committee":"query_by_committee", "Expected Error Reduction":"exp_error_reduction", "Expected Model Change":"exp_model_change"}
parameters = "(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, init_size, cost)"

for ds in tqdm(datasets,  desc ="Dataset"):
    for classifier in classifiers:
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds) # Seta aqui mesmo ou no loop do dataset?
        for idx_bag in range(n_splits):
            for key, value in modal_strategies.items():
                tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + key)
                result = modal_framework(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, [key, value])
                result['dataset'] = ds[:-5]
                total_performance_history.append(result)
                tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + key)

Dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Testando: 1043_ada_agnostic 5NN 1/5 Uncertain Sampling
uncertain_sampling


Dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Passou: 1043_ada_agnostic 5NN 1/5 Uncertain Sampling
Testando: 1043_ada_agnostic 5NN 1/5 Random Sampling
random_sampling


Dataset:   0%|          | 0/100 [00:01<?, ?it/s]

Passou: 1043_ada_agnostic 5NN 1/5 Random Sampling
Testando: 1043_ada_agnostic 5NN 1/5 Query by Committee
query_by_committee


Dataset:   0%|          | 0/100 [00:04<?, ?it/s]

Passou: 1043_ada_agnostic 5NN 1/5 Query by Committee
Testando: 1043_ada_agnostic 5NN 1/5 Expected Error Reduction
exp_error_reduction


Dataset:   0%|          | 0/100 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [19]:
total_performance_history

[{'accuracy_history': [0.7524861878453039,
   0.7524861878453039,
   0.7613259668508288,
   0.7812154696132597,
   0.7812154696132597,
   0.7823204419889502,
   0.7955801104972375,
   0.7955801104972375,
   0.7955801104972375,
   0.7878453038674034,
   0.7878453038674034],
  'f1_history': [0.6462081890575686,
   0.6462081890575686,
   0.6880516748769125,
   0.7508212632522026,
   0.7508212632522026,
   0.763228034143005,
   0.7838509513555105,
   0.7838509513555105,
   0.7838509513555105,
   0.7701031235628674,
   0.7701031235628674],
  'package': 'modAL',
  'time_elapsed': 0.6399319000001924,
  'classifier': '5NN',
  'sample_size': 0.008110477860587462,
  'strategy': 'Uncertain Sampling',
  'dataset': '1043_ada_agnostic'},
 {'accuracy_history': [0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039,
   0.7524861878453039],
  'f1_history':

## Visualization

### Save output

In [ ]:
output = pd.DataFrame.from_dict(total_performance_history)
output = output.set_index(['package','time_elapsed','classifier','sample_size','strategy','dataset']).apply(pd.Series.explode).reset_index()
output.to_csv('total_performance_history.output.csv', index=False)

In [ ]:
output

### Preprocessing

In [39]:
df = pd.DataFrame.from_dict(total_performance_history)

### Checking Strategies on Data Frame

In [40]:
pd.unique(df['strategy'])

array(['Uncertain Sampling', 'Random Sampling', 'Query by Committee'],
      dtype=object)

In [42]:
df = df.set_index(['package','time_elapsed','classifier','sample_size','strategy','dataset']).apply(pd.Series.explode).reset_index()

In [44]:
performance_mean = df.groupby(['strategy', 'classifier']).mean()
performance_std = df.groupby(['strategy', 'classifier']).std()

In [45]:
performance_mean

,,time_elapsed,sample_size
strategy,classifier,,
Query by Committee,5NN,3.453216,0.026304
Random Sampling,5NN,0.540463,0.016879
Uncertain Sampling,5NN,0.639932,0.008110


### Last loop results (last cost)

In [46]:
df_last_result = df[:]

In [48]:
df_last_result['f1_history'] = df_last_result['f1_history']
df_last_result['accuracy_history'] = df_last_result['accuracy_history']

In [50]:
df_last_result

,package,time_elapsed,classifier,sample_size,strategy,dataset,accuracy_history,f1_history
0,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.752486,0.646208
1,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.752486,0.646208
2,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.761326,0.688052
3,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.781215,0.750821
4,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.781215,0.750821
5,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.78232,0.763228
6,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.79558,0.783851
7,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.79558,0.783851
8,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.79558,0.783851
9,modAL,0.639932,5NN,0.008110,Uncertain Sampling,1043_ada_agnostic,0.787845,0.770103


In [53]:
df_last_result_mean = df_last_result.groupby(['strategy', 'classifier']).mean()
df_last_result_std = df_last_result.groupby(['strategy', 'classifier']).std()

In [54]:
df_last_result_mean

,,time_elapsed,sample_size
strategy,classifier,,
Query by Committee,5NN,3.453216,0.026304
Random Sampling,5NN,0.540463,0.016879
Uncertain Sampling,5NN,0.639932,0.008110


### Removing most time expensive strategies to improve visualization 

In [ ]:
df[df.Strategy != "Query by Committee"].sort_values('performance_history', ascending = False)

In [ ]:
df[df.Strategy == "Expected Error Reduction"].sort_values('time_elapsed', ascending = False)

In [ ]:
df.info()

### Plots

In [62]:
g = sns.relplot(
    data= df,
    x="accuracy_history", y="time_elapsed",
    hue="Strategy", size="sample_size",
    palette=sns.color_palette(n_colors=10), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

ValueError: Could not interpret value `Strategy` for parameter `hue`

In [ ]:
g = sns.relplot(
    data= df_last_result_mean,
    x="accuracy_history", y="time_elapsed",
    hue="Strategy", size="sample_size", style="classifier", height=7,
    palette=sns.color_palette(n_colors=len(pd.unique(df_last_result['Strategy']))), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

In [63]:
ax = sns.boxplot(data=df_last_result, x="accuracy_history", y="Strategy", orient="h"
                )

ValueError: Could not interpret input 'Strategy'

In [ ]:
ax = sns.boxplot(data=df_last_result, x="f1_history", y="Strategy", orient="h"
                )

In [ ]:
dfHeatMap = df_last_result.filter(['sample_size','f1_history', 'accuracy_history'], axis=1)
ax = sns.heatmap(dfHeatMap)

### Groupby por dataset

In [55]:
df = pd.DataFrame.from_dict(total_performance_history)

In [57]:
# explode pelas colunas accuracy_history e f1_history
df = df.set_index(['package','time_elapsed','classifier','sample_size','strategy','dataset']).apply(pd.Series.explode).reset_index()

# explode retorna coluna como object, logo é necessário transformar em número
df['f1_history'] = pd.to_numeric(df['f1_history'])
df['accuracy_history'] = pd.to_numeric(df['accuracy_history'])

In [60]:
df.groupby(['dataset','package','strategy', 'classifier']).mean().sort_values("f1_history", ascending = False)

time_elapsed  \
dataset           package strategy           classifier                 
1043_ada_agnostic modAL   Uncertain Sampling 5NN             0.639932   
                          Random Sampling    5NN             0.540463   
                          Query by Committee 5NN             3.453216   

                                                         sample_size  \
dataset           package strategy           classifier                
1043_ada_agnostic modAL   Uncertain Sampling 5NN            0.008110   
                          Random Sampling    5NN            0.016879   
                          Query by Committee 5NN            0.026304   

                                                         accuracy_history  \
dataset           package strategy           classifier                     
1043_ada_agnostic modAL   Uncertain Sampling 5NN                 0.779407   
                          Random Sampling    5NN                 0.752486   
                          Query by Committee 5NN                 0.751236   

                                                         f1_history  
dataset           package strategy           classifier              
1043_ada_agnostic modAL   Uncertain Sampling 5NN           0.739736  
                          Random Sampling    5NN           0.646208  
                          Query by Committee 5NN           0.644523

In [61]:
df.groupby(['dataset','package','strategy', 'classifier']).std().sort_values("f1_history", ascending = False)

time_elapsed  \
dataset           package strategy           classifier                 
1043_ada_agnostic modAL   Uncertain Sampling 5NN                  0.0   
                          Query by Committee 5NN                  0.0   
                          Random Sampling    5NN                  0.0   

                                                         sample_size  \
dataset           package strategy           classifier                
1043_ada_agnostic modAL   Uncertain Sampling 5NN                 0.0   
                          Query by Committee 5NN                 0.0   
                          Random Sampling    5NN                 0.0   

                                                         accuracy_history  \
dataset           package strategy           classifier                     
1043_ada_agnostic modAL   Uncertain Sampling 5NN                 0.016488   
                          Query by Committee 5NN                 0.000415   
                          Random Sampling    5NN                 0.000000   

                                                         f1_history  
dataset           package strategy           classifier              
1043_ada_agnostic modAL   Uncertain Sampling 5NN           0.053522  
                          Query by Committee 5NN           0.000559  
                          Random Sampling    5NN           0.000000

## Testes